In [103]:
import pandas as pd

url1 = 'https://raw.github.com/WesleyPasfield/March_Madness/master/KenPom.csv'
url2 = 'https://raw.github.com/WesleyPasfield/March_Madness/master/RegularSeasonDetailedResults.csv'
url3 = 'https://raw.github.com/WesleyPasfield/March_Madness/master/Teams.csv'
url4 = 'https://raw.github.com/WesleyPasfield/March_Madness/master/TourneyCompactResults.csv'
url5 = 'https://raw.github.com/WesleyPasfield/March_Madness/master/TourneySlots.csv'
url6 = 'https://raw.github.com/WesleyPasfield/March_Madness/master/Team_Lookup.csv'
url7 = 'https://raw.github.com/WesleyPasfield/March_Madness/master/RegularSeasonCompactResults.csv'
url8 = 'https://raw.github.com/WesleyPasfield/March_Madness/master/Stage_1.csv'
kp = pd.read_csv(url1)
regSeason = pd.read_csv(url2)
teams = pd.read_csv(url3)
tourneyResults = pd.read_csv(url4)
tourneySlots = pd.read_csv(url5)
teamLookup = pd.read_csv(url6)
regSeasonComp = pd.read_csv(url7)
stage1 = pd.read_csv(url8)

In [27]:
## Drop headers that exist in kp CSV file

kp = kp.dropna(subset = ['Pyth_NCSOS'])

## Replace seeds from team name

kp['Team'] = kp['Team'].str.replace(' 16', '')
kp['Team'] = kp['Team'].str.replace(' 15', '')
kp['Team'] = kp['Team'].str.replace(' 14', '')
kp['Team'] = kp['Team'].str.replace(' 13', '')
kp['Team'] = kp['Team'].str.replace(' 12', '')
kp['Team'] = kp['Team'].str.replace(' 11', '')
kp['Team'] = kp['Team'].str.replace(' 10', '')
kp['Team'] = kp['Team'].str.replace(' 9', '')
kp['Team'] = kp['Team'].str.replace(' 8', '')
kp['Team'] = kp['Team'].str.replace(' 7', '')
kp['Team'] = kp['Team'].str.replace(' 6', '')
kp['Team'] = kp['Team'].str.replace(' 5', '')
kp['Team'] = kp['Team'].str.replace(' 4', '')
kp['Team'] = kp['Team'].str.replace(' 3', '')
kp['Team'] = kp['Team'].str.replace(' 2', '')
kp['Team'] = kp['Team'].str.replace(' 1', '')

## Change column names for team lookup, then merge to match KP names & Kaggle Names

teamLookup.columns = ['Team', 'Team_Name']
kp2 = kp.merge(teamLookup, on = 'Team', how = 'left')

## Drop Null cases (Winston Salem St. , irrelevant), & Merge with Teams to get team_ids

kp2['nullCheck'] = pd.isnull(kp2['Team_Name'])
kp2= kp2[kp2.nullCheck != True]
kp3 = kp2.merge(teams, on = 'Team_Name', how = 'left')

## Convert variables to correct format & drop irrelevant variables

kp3[['Pyth','AdjO','AdjD','AdjT','Luck']] = kp3[['Pyth','AdjO','AdjD','AdjT','Luck']].astype(float)
kp3.drop(['Rank','Team','Conf','nullCheck','Team_Name'],inplace=True,axis=1,errors='ignore')

## Add in total adjusted team efficiency (adjO - adjD) & create variable for Team_Year

kp3['AdjT'] = kp3['AdjO'] - kp3['AdjD']
kp3['Team_Year'] = kp3.Year.map(str) + "_" + kp3.Team_Id.map(str)
kp3 = kp3[kp3['Year'] != 2002]

## Drop Unnecessary Variables

kp3.drop(['Year', 'Team_Id'], inplace=True, axis=1, errors='ignore')

## Create z score for all variables

cols = list(kp3.columns)
cols.remove('Team_Year')

for col in cols:
    col_zscore = col + '_zscore'
    kp3[col_zscore] = (kp3[col] - kp3[col].mean())/kp3[col].std(ddof=0)
    
## Pull out only z score tranformed variables
kpFin = kp3[kp3.columns[9:19]]

In [28]:
## Get Team_Year combinations, then create df for all game winners & losers

regSeason['W_Team_Year'] = regSeason.Season.map(str) + '_' + regSeason.Wteam.map(str)
regSeason['L_Team_Year'] = regSeason.Season.map(str) + '_' + regSeason.Lteam.map(str)
regSeasonFin= regSeason[['W_Team_Year', 'L_Team_Year', 'possW', 'possL', 'possDiff']]

## Create KP file that just has adjO, adjD, & AdjT for merging

kp4 = kp3[['Team_Year', 'AdjO','AdjD','AdjT']]

## Merge kp3 with regSeasonFin

regSeasonFin2 = regSeasonFin.merge(kp4, left_on = 'W_Team_Year', right_on = 'Team_Year')
regSeasonFin3 = regSeasonFin2.merge(kp4, left_on = 'L_Team_Year', right_on = 'Team_Year')
regSeasonFin3.columns = ['W_Team_Year', 'L_Team_Year', 'possW', 'possL', 
                         'possDiff', 'W_Team_Year2','W_AdjO','W_AdjD','W_AdjT',
                        'L_Team_Year2', 'L_AdjO', 'L_AdjD', 'L_AdjT']

regSeasonFin4 = regSeasonFin3
regSeasonFin4['W_expO'] = (regSeasonFin3['W_AdjO'] + regSeasonFin3['L_AdjD']) / 2
regSeasonFin4['W_expD'] = (regSeasonFin3['W_AdjD'] + regSeasonFin3['L_AdjO']) / 2
regSeasonFin4['L_expO'] = (regSeasonFin3['L_AdjO'] + regSeasonFin3['W_AdjD']) / 2
regSeasonFin4['L_expD'] = (regSeasonFin3['L_AdjD'] + regSeasonFin3['W_AdjO']) / 2
regSeasonFin4['expF'] = regSeasonFin3['W_AdjT'] - regSeasonFin3['L_AdjT']
regSeasonFin4['W_ActO_ExpO'] = regSeasonFin4['possW'] - regSeasonFin4['W_expO']
regSeasonFin4['W_ActD_ExpD'] = regSeasonFin4['possL'] - regSeasonFin4['W_expD']
regSeasonFin4['L_ActO_ExpO'] = regSeasonFin4['possL'] - regSeasonFin4['L_expO']
regSeasonFin4['L_ActD_ExpD'] = regSeasonFin4['possW'] - regSeasonFin4['L_expD']
regSeasonFin4['ActF-ExpF'] = regSeasonFin4['possDiff'] - regSeasonFin4['expF']
regSeasonFin4['W_ActO_ExpO_Abs'] = abs(regSeasonFin4['possW'] - regSeasonFin4['W_expO'])
regSeasonFin4['W_ActD_ExpD_Abs'] = abs(regSeasonFin4['possL'] - regSeasonFin4['W_expD'])
regSeasonFin4['L_ActO_ExpO_Abs'] = abs(regSeasonFin4['possL'] - regSeasonFin4['L_expO'])
regSeasonFin4['L_ActD_ExpD_Abs'] = abs(regSeasonFin4['possW'] - regSeasonFin4['L_expD'])
regSeasonFin4['ActF-ExpF_Abs'] = abs(regSeasonFin4['possDiff'] - regSeasonFin4['expF'])

regSeasLoss = regSeasonFin4[['L_Team_Year','L_ActO_ExpO', 'L_ActO_ExpO_Abs', 
                             'L_ActD_ExpD','L_ActD_ExpD_Abs','ActF-ExpF','ActF-ExpF_Abs']]
                                                            
regSeasWin = regSeasonFin4[['W_Team_Year','W_ActO_ExpO', 'W_ActO_ExpO_Abs', 'W_ActD_ExpD',
                             'W_ActD_ExpD_Abs', 'ActF-ExpF', 'ActF-ExpF_Abs']]

regSeasLoss.columns = ['Team_Year', 'AdjO_Vol', 'AdjO_Vol_Abs', 'AdjD_Vol', 'AdjD_Vol_Abs', 'AdjT_Vol', 'AdjT_Vol_Abs']
regSeasWin.columns = ['Team_Year', 'AdjO_Vol', 'AdjO_Vol_Abs', 'AdjD_Vol', 'AdjD_Vol_Abs', 'AdjT_Vol', 'AdjT_Vol_Abs']

regSeasConcat = pd.concat([regSeasLoss, regSeasWin])
regSeasGroupAverage = regSeasConcat.groupby(['Team_Year']).mean()
regSeasGroupAverage['Team_Year'] = regSeasGroupAverage.index
regSeasGroupAverage = regSeasGroupAverage[['Team_Year', 'AdjO_Vol_Abs', 'AdjD_Vol_Abs', 'AdjT_Vol_Abs']]
regSeasGroupAverage.columns = ['Team_Year', 'AdjO_Vol_Avg', 'AdjD_Vol_Avg', 'AdjT_Vol_Avg']

regSeasGroupMax = regSeasConcat.groupby(['Team_Year']).max()
regSeasGroupMax['Team_Year'] = regSeasGroupMax.index
regSeasGroupMax = regSeasGroupMax[['Team_Year', 'AdjO_Vol' ,'AdjD_Vol', 'AdjT_Vol']]
regSeasGroupMax.columns = ['Team_Year', 'AdjO_Vol_Max' ,'AdjD_Vol_Max', 'AdjT_Vol_Max']

regSeasGroupMin = regSeasConcat.groupby(['Team_Year']).min()
regSeasGroupMin['Team_Year'] = regSeasGroupMin.index
regSeasGroupMin = regSeasGroupMin[['Team_Year', 'AdjO_Vol' ,'AdjD_Vol', 'AdjT_Vol']]
regSeasGroupMin.columns = ['Team_Year', 'AdjO_Vol_Min' ,'AdjD_Vol_Min', 'AdjT_Vol_Min']

regSeasGroupMaxMin = regSeasGroupMax.merge(regSeasGroupMin, on = 'Team_Year')
regSeasGroupMaxMin['AdjO_Vol_Range'] = regSeasGroupMaxMin['AdjO_Vol_Max'] - regSeasGroupMaxMin['AdjO_Vol_Min']
regSeasGroupMaxMin['AdjD_Vol_Range'] = regSeasGroupMaxMin['AdjD_Vol_Max'] - regSeasGroupMaxMin['AdjD_Vol_Min']
regSeasGroupMaxMin['AdjT_Vol_Range'] = regSeasGroupMaxMin['AdjT_Vol_Max'] - regSeasGroupMaxMin['AdjT_Vol_Min']
regSeasGroupFin = regSeasGroupMaxMin.merge(regSeasGroupAverage, on = 'Team_Year')
regSeasGroupFin.index = regSeasGroupFin['Team_Year']
regSeasGroupFin.head(n =10)

## Create z score for all variables

cols = list(regSeasGroupFin.columns)
cols.remove('Team_Year')

for col in cols:
    col_zscore = col + '_zscore'
    regSeasGroupFin[col_zscore] = (regSeasGroupFin[col] - regSeasGroupFin[col].mean())/regSeasGroupFin[col].std(ddof=0)
    
regSeasMerge = regSeasGroupFin[['Team_Year', 'AdjO_Vol_Max_zscore', 'AdjD_Vol_Max_zscore',
                               'AdjT_Vol_Max_zscore', 'AdjO_Vol_Min_zscore', 'AdjD_Vol_Min_zscore',
                               'AdjT_Vol_Min_zscore', 'AdjO_Vol_Range_zscore', 'AdjD_Vol_Range_zscore',
                               'AdjT_Vol_Range_zscore', 'AdjO_Vol_Avg_zscore', 'AdjD_Vol_Avg_zscore',
                               'AdjT_Vol_Avg_zscore']]


kpMerge = kpFin.merge(regSeasMerge, on = 'Team_Year', how = 'left')
kpMerge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4420 entries, 0 to 4419
Data columns (total 22 columns):
Team_Year                4420 non-null object
Pyth_zscore              4420 non-null float64
AdjO_zscore              4420 non-null float64
AdjD_zscore              4420 non-null float64
AdjT_zscore              4420 non-null float64
Luck_zscore              4420 non-null float64
Pyth_SOS_zscore          4420 non-null float64
OppO_zscore              4420 non-null float64
OppD_zscore              4420 non-null float64
Pyth_NCSOS_zscore        4420 non-null float64
AdjO_Vol_Max_zscore      4420 non-null float64
AdjD_Vol_Max_zscore      4420 non-null float64
AdjT_Vol_Max_zscore      4420 non-null float64
AdjO_Vol_Min_zscore      4420 non-null float64
AdjD_Vol_Min_zscore      4420 non-null float64
AdjT_Vol_Min_zscore      4420 non-null float64
AdjO_Vol_Range_zscore    4420 non-null float64
AdjD_Vol_Range_zscore    4420 non-null float64
AdjT_Vol_Range_zscore    4420 non-null float64
A

In [41]:

regUse = regSeasonComp[['Wteam', 'Lteam', 'Home', 'Neutral', 'Away', 'Win']]

kpModel = regUse.merge(kpMerge, left_on = 'Wteam', right_on = 'Team_Year')
kpModel = kpModel.merge(kpMerge, left_on = 'Lteam', right_on = 'Team_Year')

kpModel['Pyth_diff'] = kpModel['Pyth_zscore_x'] - kpModel['Pyth_zscore_y']
kpModel['AdjO_diff'] = kpModel['AdjO_zscore_x'] - kpModel['AdjO_zscore_y']
kpModel['AdjD_diff'] = kpModel['AdjD_zscore_x'] - kpModel['AdjD_zscore_y']
kpModel['AdjT_diff'] = kpModel['AdjT_zscore_x'] - kpModel['AdjT_zscore_y']
kpModel['Luck_diff'] = kpModel['Luck_zscore_x'] - kpModel['Luck_zscore_y']
kpModel['Pyth_SOS_diff'] = kpModel['Pyth_SOS_zscore_x'] - kpModel['Pyth_SOS_zscore_y']
kpModel['OppO_diff'] = kpModel['OppO_zscore_x'] - kpModel['OppO_zscore_y']
kpModel['OppD_diff'] = kpModel['OppD_zscore_x'] - kpModel['OppD_zscore_y']
kpModel['Pyth_NCSOS_diff'] = kpModel['Pyth_NCSOS_zscore_x'] - kpModel['Pyth_NCSOS_zscore_y']
kpModel['AdjO_Vol_Max_diff'] = kpModel['AdjO_Vol_Max_zscore_x'] - kpModel['AdjO_Vol_Max_zscore_y']
kpModel['AdjD_Vol_Max_diff'] = kpModel['AdjD_Vol_Max_zscore_x'] - kpModel['AdjD_Vol_Max_zscore_y']
kpModel['AdjT_Vol_Max_diff'] = kpModel['AdjT_Vol_Max_zscore_x'] - kpModel['AdjT_Vol_Max_zscore_y']
kpModel['AdjO_Vol_Min_diff'] = kpModel['AdjO_Vol_Min_zscore_x'] - kpModel['AdjO_Vol_Min_zscore_y']
kpModel['AdjD_Vol_Min_diff'] = kpModel['AdjD_Vol_Min_zscore_x'] - kpModel['AdjD_Vol_Min_zscore_y']
kpModel['AdjT_Vol_Min_diff'] = kpModel['AdjT_Vol_Min_zscore_x'] - kpModel['AdjT_Vol_Min_zscore_y']
kpModel['AdjO_Vol_Range_diff'] = kpModel['AdjO_Vol_Range_zscore_x'] - kpModel['AdjO_Vol_Range_zscore_y']
kpModel['AdjD_Vol_Range_diff'] = kpModel['AdjD_Vol_Range_zscore_x'] - kpModel['AdjD_Vol_Range_zscore_y']
kpModel['AdjT_Vol_Range_diff'] = kpModel['AdjT_Vol_Range_zscore_x'] - kpModel['AdjT_Vol_Range_zscore_y']
kpModel['AdjO_Vol_Avg_diff'] = kpModel['AdjO_Vol_Avg_zscore_x'] - kpModel['AdjO_Vol_Avg_zscore_y']
kpModel['AdjD_Vol_Avg_diff'] = kpModel['AdjD_Vol_Avg_zscore_x'] - kpModel['AdjD_Vol_Avg_zscore_y']
kpModel['AdjT_Vol_Avg_diff'] = kpModel['AdjT_Vol_Avg_zscore_x'] - kpModel['AdjT_Vol_Avg_zscore_y']

kpModel['id'] = kpModel.Wteam.str[:4] + '_' + kpModel.Wteam.str[4:] + '_' + kpModel.Lteam.str[4:]
#kp3['Team_Year'] = kp3.Year.map(str) + "_" + kp3.Team_Id.map(str)
kpFinal = kpModel.iloc[:,[2,3,4,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70]]
kpTarget = kpModel['Win']

In [51]:

import sklearn.ensemble as ens
from sklearn.preprocessing import LabelEncoder
import numpy as np

tIndex = np.random.rand(len(kpFinal)) < 0.75
X_train = kpFinal[tIndex]
X_test = kpFinal[~tIndex]
y_train = kpTarget[tIndex]
y_test = kpTarget[~tIndex]

gbm = ens.GradientBoostingClassifier(max_depth =5 , learning_rate = 0.1, n_estimators = 100,
                                    subsample= .85).fit(X_train, y_train)




array([0, 0, 0, ..., 1, 0, 0])

In [57]:
predictions = gbm.predict_proba(X_test)
predictions


array([[ 0.69273905,  0.30726095],
       [ 0.62813065,  0.37186935],
       [ 0.6898363 ,  0.3101637 ],
       ..., 
       [ 0.13453458,  0.86546542],
       [ 0.69809482,  0.30190518],
       [ 0.8417771 ,  0.1582229 ]])

In [61]:
import xgboost as xgb

xgb1 = xgb.XGBClassifier(max_depth =5 , learning_rate = 0.1, n_estimators = 100,
                                    subsample= .85, colsample_bytree =.85, 
                                     objective = 'binary:logistic').fit(X_train, y_train)


In [62]:
predictions = xgb1.predict_proba(X_test)
predictions

array([[ 0.68761265,  0.31238735],
       [ 0.58896637,  0.41103363],
       [ 0.67596853,  0.3240315 ],
       ..., 
       [ 0.18509209,  0.81490791],
       [ 0.70939124,  0.29060876],
       [ 0.8305701 ,  0.16942993]], dtype=float32)

In [82]:
## Create modeling file by merging reg season results with tourney results

tourneyUse = tourneyResults[['Wteam', 'Lteam', 'Home', 'Neutral', 'Away', 'Win']]

kpModelTourn = tourneyUse.merge(kpMerge, left_on = 'Wteam', right_on = 'Team_Year')
kpModelTourn = kpModelTourn.merge(kpMerge, left_on = 'Lteam', right_on = 'Team_Year')

kpModelTourn['Pyth_diff'] = kpModelTourn['Pyth_zscore_x'] - kpModelTourn['Pyth_zscore_y']
kpModelTourn['AdjO_diff'] = kpModelTourn['AdjO_zscore_x'] - kpModelTourn['AdjO_zscore_y']
kpModelTourn['AdjD_diff'] = kpModelTourn['AdjD_zscore_x'] - kpModelTourn['AdjD_zscore_y']
kpModelTourn['AdjT_diff'] = kpModelTourn['AdjT_zscore_x'] - kpModelTourn['AdjT_zscore_y']
kpModelTourn['Luck_diff'] = kpModelTourn['Luck_zscore_x'] - kpModelTourn['Luck_zscore_y']
kpModelTourn['Pyth_SOS_diff'] = kpModelTourn['Pyth_SOS_zscore_x'] - kpModelTourn['Pyth_SOS_zscore_y']
kpModelTourn['OppO_diff'] = kpModelTourn['OppO_zscore_x'] - kpModelTourn['OppO_zscore_y']
kpModelTourn['OppD_diff'] = kpModelTourn['OppD_zscore_x'] - kpModelTourn['OppD_zscore_y']
kpModelTourn['Pyth_NCSOS_diff'] = kpModelTourn['Pyth_NCSOS_zscore_x'] - kpModelTourn['Pyth_NCSOS_zscore_y']
kpModelTourn['AdjO_Vol_Max_diff'] = kpModelTourn['AdjO_Vol_Max_zscore_x'] - kpModelTourn['AdjO_Vol_Max_zscore_y']
kpModelTourn['AdjD_Vol_Max_diff'] = kpModelTourn['AdjD_Vol_Max_zscore_x'] - kpModelTourn['AdjD_Vol_Max_zscore_y']
kpModelTourn['AdjT_Vol_Max_diff'] = kpModelTourn['AdjT_Vol_Max_zscore_x'] - kpModelTourn['AdjT_Vol_Max_zscore_y']
kpModelTourn['AdjO_Vol_Min_diff'] = kpModelTourn['AdjO_Vol_Min_zscore_x'] - kpModelTourn['AdjO_Vol_Min_zscore_y']
kpModelTourn['AdjD_Vol_Min_diff'] = kpModelTourn['AdjD_Vol_Min_zscore_x'] - kpModelTourn['AdjD_Vol_Min_zscore_y']
kpModelTourn['AdjT_Vol_Min_diff'] = kpModelTourn['AdjT_Vol_Min_zscore_x'] - kpModelTourn['AdjT_Vol_Min_zscore_y']
kpModelTourn['AdjO_Vol_Range_diff'] = kpModelTourn['AdjO_Vol_Range_zscore_x'] - kpModelTourn['AdjO_Vol_Range_zscore_y']
kpModelTourn['AdjD_Vol_Range_diff'] = kpModelTourn['AdjD_Vol_Range_zscore_x'] - kpModelTourn['AdjD_Vol_Range_zscore_y']
kpModelTourn['AdjT_Vol_Range_diff'] = kpModelTourn['AdjT_Vol_Range_zscore_x'] - kpModelTourn['AdjT_Vol_Range_zscore_y']
kpModelTourn['AdjO_Vol_Avg_diff'] = kpModelTourn['AdjO_Vol_Avg_zscore_x'] - kpModelTourn['AdjO_Vol_Avg_zscore_y']
kpModelTourn['AdjD_Vol_Avg_diff'] = kpModelTourn['AdjD_Vol_Avg_zscore_x'] - kpModelTourn['AdjD_Vol_Avg_zscore_y']
kpModelTourn['AdjT_Vol_Avg_diff'] = kpModelTourn['AdjT_Vol_Avg_zscore_x'] - kpModelTourn['AdjT_Vol_Avg_zscore_y']

kpModelTourn['id'] = kpModelTourn.Wteam.str[:4] + '_' + kpModelTourn.Wteam.str[4:] + '_' + kpModelTourn.Lteam.str[4:]
#kp3['Team_Year'] = kp3.Year.map(str) + "_" + kp3.Team_Id.map(str)
kpFinalTourn = kpModelTourn.iloc[:,[2,3,4,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70]]
kpTarget = kpModelTourn['Win']

In [113]:
## Test Predictor


kpModelStage1 = stage1.merge(kpMerge, left_on = 'Wteam', right_on = 'Team_Year')
kpModelStage1 = kpModelStage1.merge(kpMerge, left_on = 'Lteam', right_on = 'Team_Year')

kpModelStage1['Pyth_diff'] = kpModelStage1['Pyth_zscore_x'] - kpModelStage1['Pyth_zscore_y']
kpModelStage1['AdjO_diff'] = kpModelStage1['AdjO_zscore_x'] - kpModelStage1['AdjO_zscore_y']
kpModelStage1['AdjD_diff'] = kpModelStage1['AdjD_zscore_x'] - kpModelStage1['AdjD_zscore_y']
kpModelStage1['AdjT_diff'] = kpModelStage1['AdjT_zscore_x'] - kpModelStage1['AdjT_zscore_y']
kpModelStage1['Luck_diff'] = kpModelStage1['Luck_zscore_x'] - kpModelStage1['Luck_zscore_y']
kpModelStage1['Pyth_SOS_diff'] = kpModelStage1['Pyth_SOS_zscore_x'] - kpModelStage1['Pyth_SOS_zscore_y']
kpModelStage1['OppO_diff'] = kpModelStage1['OppO_zscore_x'] - kpModelStage1['OppO_zscore_y']
kpModelStage1['OppD_diff'] = kpModelStage1['OppD_zscore_x'] - kpModelStage1['OppD_zscore_y']
kpModelStage1['Pyth_NCSOS_diff'] = kpModelStage1['Pyth_NCSOS_zscore_x'] - kpModelStage1['Pyth_NCSOS_zscore_y']
kpModelStage1['AdjO_Vol_Max_diff'] = kpModelStage1['AdjO_Vol_Max_zscore_x'] - kpModelStage1['AdjO_Vol_Max_zscore_y']
kpModelStage1['AdjD_Vol_Max_diff'] = kpModelStage1['AdjD_Vol_Max_zscore_x'] - kpModelStage1['AdjD_Vol_Max_zscore_y']
kpModelStage1['AdjT_Vol_Max_diff'] = kpModelStage1['AdjT_Vol_Max_zscore_x'] - kpModelStage1['AdjT_Vol_Max_zscore_y']
kpModelStage1['AdjO_Vol_Min_diff'] = kpModelStage1['AdjO_Vol_Min_zscore_x'] - kpModelStage1['AdjO_Vol_Min_zscore_y']
kpModelStage1['AdjD_Vol_Min_diff'] = kpModelStage1['AdjD_Vol_Min_zscore_x'] - kpModelStage1['AdjD_Vol_Min_zscore_y']
kpModelStage1['AdjT_Vol_Min_diff'] = kpModelStage1['AdjT_Vol_Min_zscore_x'] - kpModelStage1['AdjT_Vol_Min_zscore_y']
kpModelStage1['AdjO_Vol_Range_diff'] = kpModelStage1['AdjO_Vol_Range_zscore_x'] - kpModelStage1['AdjO_Vol_Range_zscore_y']
kpModelStage1['AdjD_Vol_Range_diff'] = kpModelStage1['AdjD_Vol_Range_zscore_x'] - kpModelStage1['AdjD_Vol_Range_zscore_y']
kpModelStage1['AdjT_Vol_Range_diff'] = kpModelStage1['AdjT_Vol_Range_zscore_x'] - kpModelStage1['AdjT_Vol_Range_zscore_y']
kpModelStage1['AdjO_Vol_Avg_diff'] = kpModelStage1['AdjO_Vol_Avg_zscore_x'] - kpModelStage1['AdjO_Vol_Avg_zscore_y']
kpModelStage1['AdjD_Vol_Avg_diff'] = kpModelStage1['AdjD_Vol_Avg_zscore_x'] - kpModelStage1['AdjD_Vol_Avg_zscore_y']
kpModelStage1['AdjT_Vol_Avg_diff'] = kpModelStage1['AdjT_Vol_Avg_zscore_x'] - kpModelStage1['AdjT_Vol_Avg_zscore_y']

kpModelStage1['id'] = kpModelStage1.Wteam.str[:4] + '_' + kpModelStage1.Wteam.str[4:] + '_' + kpModelStage1.Lteam.str[4:]
kpFinalStage1 = kpModelStage1.iloc[:,[2,3,4,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69]]


In [114]:
kpFinalStage1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9112 entries, 0 to 9111
Data columns (total 24 columns):
Home                   9112 non-null int64
Neutral                9112 non-null int64
Away                   9112 non-null int64
Pyth_diff              9112 non-null float64
AdjO_diff              9112 non-null float64
AdjD_diff              9112 non-null float64
AdjT_diff              9112 non-null float64
Luck_diff              9112 non-null float64
Pyth_SOS_diff          9112 non-null float64
OppO_diff              9112 non-null float64
OppD_diff              9112 non-null float64
Pyth_NCSOS_diff        9112 non-null float64
AdjO_Vol_Max_diff      9112 non-null float64
AdjD_Vol_Max_diff      9112 non-null float64
AdjT_Vol_Max_diff      9112 non-null float64
AdjO_Vol_Min_diff      9112 non-null float64
AdjD_Vol_Min_diff      9112 non-null float64
AdjT_Vol_Min_diff      9112 non-null float64
AdjO_Vol_Range_diff    9112 non-null float64
AdjD_Vol_Range_diff    9112 non-null float64

In [118]:
import os


X_testStage1 = kpFinalStage1

predSub = gbm.predict_proba(X_testStage1)
predSub2 = xgb1.predict_proba(X_testStage1)

submission = pd.DataFrame({'id': kpModelStage1['id'],
                           'pred' : predSub[:,1]})
submission2 = pd.DataFrame({'id': kpModelStage1['id'],
                           'pred' : predSub2[:,1]})

path = '/Users/Wesley/Documents/Kaggle/MarchMadness'
os.chdir(path)

submission.to_csv('gbmSubmission.csv')
submission2.to_csv('xgbSubmission.csv')



In [95]:
from matplotlib import pylab as plt
xgb.plot_importance(xgb1)
plt.figure()
df.plot()
df.plot(kind='barh', x='feature', y='fscore', legend=False, figsize=(6, 10))
plt.title('XGBoost Feature Importance')
plt.xlabel('relative importance')
plt.gcf().savefig('feature_importance_xgb.png')

NameError: name 'df' is not defined

In [97]:
TO DO

Visualize the data
Tune the model
Work in Tourney results
Remove __ from id variable, not sure why that is happening

NameError: name 'getcwd' is not defined